In [1]:
#groupby and fillna of competition
def ad_fuller(col_values):
    result = adfuller(col_values)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
import fbprophet

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

#model building
from sklearn.model_selection import train_test_split,KFold,cross_val_score,StratifiedKFold,GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression,ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split,KFold,GridSearchCV,cross_val_score
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import lightgbm 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier


pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import warnings
warnings.filterwarnings('ignore')

import gc
gc.collect()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning:

Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.



In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train['Course_ID'] = train['Course_ID'].astype('object')
train['Day_No'] = train['Day_No'].astype('object')

In [4]:
train.tail()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,User_Traffic,Competition_Metric,Sales
512082,548023,878,600,Software Marketing,Program,0,0,1,8904,0.07,114
512083,548024,879,600,Software Marketing,Program,0,0,1,10542,0.07,145
512084,548025,880,600,Software Marketing,Program,0,0,1,13671,0.07,167
512085,548026,881,600,Software Marketing,Program,0,0,1,8904,0.07,107
512086,548027,882,600,Software Marketing,Program,1,0,1,11445,0.07,152


In [5]:
test.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric
0,883,883,1,Development,Course,1,0,1,0.007
1,884,884,1,Development,Course,1,0,1,0.007
2,885,885,1,Development,Course,1,0,1,0.007
3,886,886,1,Development,Course,1,0,1,0.007
4,887,887,1,Development,Course,0,0,1,0.007


In [6]:
train.shape,test.shape

((512087, 11), (36000, 9))

In [7]:
train['Competition_Metric'].fillna(train['Competition_Metric'].mean(),inplace=True)
test['Competition_Metric'].fillna(test['Competition_Metric'].mean(),inplace=True)

In [8]:
#using the last 60 days user traffic 

# t = train.copy()
# t = t[['Day_No','Course_ID','User_Traffic']]

# test['Day_No_temp'] = test['Day_No']-60

# test = pd.merge(test,t,left_on=['Day_No_temp','Course_ID'],right_on=['Day_No','Course_ID'],how='left')

# test.drop(['Day_No_temp','Day_No_y'],axis=1,inplace=True)
# test.rename(columns={'Day_No_x':'Day_No'},inplace=True)
# test.head()



In [9]:
# # USing prophet to model user traffic

# df = train.groupby(['Day_No','Course_ID']).agg({\
#                                    'User_Traffic':{\
#                                            "user_traffic_mean":'first',
#                                            }
#                                    }).reset_index().fillna(0)
# df.columns= ['Day_No','Course_ID','user_traffic_mean']


# temp = pd.DataFrame(columns=['Day_No'])
# n =test.Day_No.max()-test.Day_No.min()+1
# temp['Day_No']=np.arange(test.Day_No.min(),test.Day_No.max()+1)

# vals = list(df.Course_ID.unique())
# i=0
# for val in vals:
#     temp[str(val)]=np.nan
    
#     df_temp=df[df['Course_ID']==val]
#     df_temp['ds'] = pd.date_range(start='1/1/2018', periods=len(df_temp))
#     df_temp.rename(columns={'user_traffic_mean': 'y'},inplace=True)

#     df_temp=df_temp[["ds",'y']]
#     df_temp['y'] = np.where(df_temp['y']==0,1,df_temp['y'])
#     df_temp['ds'] = pd.to_datetime(df_temp['ds'])

#     df_temp_prophet = fbprophet.Prophet(    daily_seasonality=False,
#                                         weekly_seasonality=True,
#                                         yearly_seasonality=False,
#     #                                     seasonality_mode="multiplicative",
#                                         changepoint_prior_scale=0.05
#                                   )
#     df_temp_prophet.fit(df_temp)
#     df_temp_forecast = df_temp_prophet.make_future_dataframe(periods=n, freq='D')
    
#     pred = df_temp_prophet.predict(df_temp_forecast)['yhat'].values

#     temp[str(val)]=pred[-n:]
#     i+=1
#     print(i)
    
    
    

# t = temp.melt(id_vars=["Day_No"], 
#               var_name="Course_ID",\
#               value_name="user_traffic_id_fbprophet")
# t['Day_No'] = t['Day_No'] .astype(int)
# t['Course_ID'] = t['Course_ID'].astype(int)
# t.head()

# test = pd.merge(test,t,on=['Course_ID','Day_No'],how='left')
# test.rename(columns={'user_traffic_id_fbprophet':'User_Traffic'},inplace=True)    

In [10]:
# #USing moving average for USer  traffic

# final = pd.DataFrame(columns=['Day_No'])
# final['Day_No'] = np.arange(1,test.Day_No.max()+1)

# vals = list(train.Course_ID.unique())
# for val in vals: 
#     df = train.copy()
#     df=df[df['Course_ID']==val]
#     df=df[['Day_No','User_Traffic']]

#     df1 = test.copy()
#     df1=df1[df1['Course_ID']==val]
#     df1=df1[['Day_No']]
#     df1['User_Traffic']=0

#     temp = pd.concat([df,df1],ignore_index=True)


#     t = train[train['Course_ID']==val]
#     t['ma'] = t['User_Traffic'].rolling(window=60).mean()
#     t.dropna(subset=['ma'],inplace=True)
#     ma_60 = t['ma'].mean()
    
#     for  i in  range(len(temp)):
#         if (temp.iloc[i,1]==0):
            
#             fill = (60*ma_60)-(sum(temp.iloc[i-59:i,1]))
#             if fill > 0:
#                 temp.iloc[i,1]= fill
#             else:
#                 temp.iloc[i,1]= ma_60
#         else:
#             pass
#     temp.rename(columns={'User_Traffic':str(val)},inplace=True) 
    
#     final = pd.merge(final,temp,on=['Day_No'],how='left')
#     del(temp,t)


# t = final.melt(id_vars=["Day_No"], 
#               var_name="Course_ID",\
#               value_name="user_traffic_id_movingavg")
# t['Day_No'] = t['Day_No'] .astype(int)
# t['Course_ID'] = t['Course_ID'].astype(int)
# t.dropna(inplace=True)


# test = pd.merge(test,t,on=['Course_ID','Day_No'],how='left')
# test.rename(columns={'user_traffic_id_movingavg':'User_Traffic'},inplace=True) 
# test.head()

In [11]:
train.groupby(['Public_Holiday'])['Sales'].describe()

,count,mean,std,min,25%,50%,75%,max
Public_Holiday,,,,,,,,
0,495885.0,122.153818,53.893176,0.0,85.0,112.0,147.0,682.0
1,16202.0,80.215467,52.748521,0.0,38.0,78.0,110.0,441.0


In [12]:
train.groupby(['Public_Holiday','Course_Domain'])["Sales"].agg('sum').reset_index()

,Public_Holiday,Course_Domain,Sales
0,0,Business,795310
1,0,Development,31092934
2,0,Finance & Accounting,9412862
3,0,Software Marketing,19273140
4,1,Business,27565
5,1,Development,626798
6,1,Finance & Accounting,201345
7,1,Software Marketing,443943


In [13]:
cols =['ID','Day_No','Course_ID','Course_Domain','Course_Type']
data = pd.concat([train[cols],test[cols]])
data.index = np.arange(0,len(data))
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type
0,1,1,1,Development,Course
1,2,2,1,Development,Course
2,3,3,1,Development,Course
3,4,4,1,Development,Course
4,5,5,1,Development,Course


In [14]:
arima_df = pd.DataFrame(columns=['Day_No'])
arima_df['Day_No']=np.arange(1,data.Day_No.max()+1)

df = train.groupby(['Day_No']).agg({\
                                   'User_Traffic':{\
                                           "user_traffic_mean":'mean',
                                            "user_traffic_std":'std'
                                           },
                                   }).reset_index()
df.columns= ['Day_No','user_traffic_mean','user_traffic_std']
df.head()

model = ARIMA(df.user_traffic_mean,order=(3,0,3))
results_arima = model.fit()
pred=results_arima.predict(start=0,end=data.Day_No.max()+1)
arima_df['day_user_traffic_mean_arima']=round(pred,0)
#print("rmse",np.sqrt(sum((df.user_traffic_mean-results_arima.fittedvalues)**2)))
#for user_traffic mean arima==(3,0,0)

model = ARIMA(df.user_traffic_std,order=(2,0,0))
results_arima = model.fit()
pred=results_arima.predict(start=0,end=data.Day_No.max()+1)
arima_df['day_user_traffic_std_arima']=round(pred,0)
#print("rmse",np.sqrt(sum((df.user_traffic_std-results_arima.fittedvalues)**2)))
#for user_traffic mean arima==(2,0,0)
arima_df.head()

arima_df['Day_No']=arima_df['Day_No'].astype(int)

data = pd.merge(data,arima_df,on=['Day_No'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_user_traffic_mean_arima,day_user_traffic_std_arima
0,1,1,1,Development,Course,15398.0,7242.0
1,2,2,1,Development,Course,15460.0,7258.0
2,3,3,1,Development,Course,15454.0,7333.0
3,4,4,1,Development,Course,15152.0,7229.0
4,5,5,1,Development,Course,15261.0,6956.0


In [15]:
df = train.groupby(['Day_No','Course_Domain']).agg({\
                                   'User_Traffic':{\
                                           "user_traffic_mean":'mean',
                                            "user_traffic_std":'std'
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_Domain','user_traffic_mean','user_traffic_std']

temp_df = data.copy()
temp_df = temp_df.groupby(["Day_No",'Course_Domain']).size().reset_index()
temp_df.columns=["Day_No",'Course_Domain','remove']
temp_df.drop(['remove'],axis=1,inplace=True)


vals = list(temp_df.Course_Domain.unique())
for val in vals:
    temp = pd.DataFrame(columns=['Day_No'])
    temp['Day_No']=np.arange(1,data.Day_No.max()+1)
    temp['Course_Domain']=val
    
    model = ARIMA(df[df["Course_Domain"]==val].user_traffic_mean,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["user_traffic_dom_mean_arima_"+str(val)]=round(pred,0)
    
    model = ARIMA(df[df["Course_Domain"]==val].user_traffic_std,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["user_traffic_dom_std_arima_"+str(val)]=round(pred,0)
    
    temp_df = pd.merge(temp_df,temp,on=['Day_No','Course_Domain'],how='outer')
    
temp_df['user_traffic_dom_mean_arima_Business'].fillna(temp_df['user_traffic_dom_mean_arima_Development'],inplace=True)
temp_df['user_traffic_dom_mean_arima_Business'].fillna(temp_df['user_traffic_dom_mean_arima_Finance & Accounting'],inplace=True)
temp_df['user_traffic_dom_mean_arima_Business'].fillna(temp_df['user_traffic_dom_mean_arima_Software Marketing'],inplace=True)

temp_df['user_traffic_dom_std_arima_Business'].fillna(temp_df['user_traffic_dom_std_arima_Development'],inplace=True)
temp_df['user_traffic_dom_std_arima_Business'].fillna(temp_df['user_traffic_dom_std_arima_Finance & Accounting'],inplace=True)
temp_df['user_traffic_dom_std_arima_Business'].fillna(temp_df['user_traffic_dom_std_arima_Software Marketing'],inplace=True)

temp_df.drop(['user_traffic_dom_mean_arima_Development','user_traffic_dom_mean_arima_Finance & Accounting','user_traffic_dom_mean_arima_Software Marketing',\
        'user_traffic_dom_std_arima_Development','user_traffic_dom_std_arima_Finance & Accounting','user_traffic_dom_std_arima_Software Marketing'],\
       axis=1,inplace=True)

temp_df.rename(columns={"user_traffic_dom_mean_arima_Business":'user_traffic_dom_mean_arima',\
                  "user_traffic_dom_std_arima_Business":"user_traffic_dom_std_arima"},inplace=True)
temp_df.head()

# train = pd.merge(train,temp_df[["Day_No","Course_Domain","user_traffic_dom_mean_arima","user_traffic_dom_std_arima"]],on=['Day_No','Course_Domain'],how='left')
# test = pd.merge(test,temp_df[["Day_No","Course_Domain","user_traffic_dom_mean_arima","user_traffic_dom_std_arima"]],on=['Day_No','Course_Domain'],how='left')
data = pd.merge(data,temp_df[["Day_No","Course_Domain","user_traffic_dom_mean_arima"]],on=['Day_No','Course_Domain'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima
0,1,1,1,Development,Course,15398.0,7242.0,16214.0
1,2,2,1,Development,Course,15460.0,7258.0,16098.0
2,3,3,1,Development,Course,15454.0,7333.0,16370.0
3,4,4,1,Development,Course,15152.0,7229.0,15934.0
4,5,5,1,Development,Course,15261.0,6956.0,16144.0


In [16]:
df = train.groupby(['Day_No','Course_Type']).agg({\
                                   'User_Traffic':{\
                                           "user_traffic_mean":'mean',
                                            "user_traffic_std":'std'
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_Type','user_traffic_mean','user_traffic_std']
df.head()

temp_df = data.copy()
temp_df = temp_df.groupby(["Day_No",'Course_Type']).size().reset_index()
temp_df.columns=["Day_No",'Course_Type','remove']
temp_df.drop(['remove'],axis=1,inplace=True)

vals = list(df.Course_Type.unique())
for val in vals:
    temp = pd.DataFrame(columns=['Day_No'])
    temp['Day_No']=np.arange(1,data.Day_No.max()+1)
    temp['Course_Type']=val
    
    model = ARIMA(df[df["Course_Type"]==val].user_traffic_mean,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["user_traffic_ctype_mean_arima_"+str(val)]=round(pred,0)
    
    model = ARIMA(df[df["Course_Type"]==val].user_traffic_std,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["user_traffic_ctype_std_arima_"+str(val)]=round(pred,0)
    
    temp_df = pd.merge(temp_df,temp,on=['Day_No','Course_Type'],how='left')

    
temp_df['user_traffic_ctype_mean_arima_Course'].fillna(temp_df['user_traffic_ctype_mean_arima_Degree'],inplace=True)
temp_df['user_traffic_ctype_mean_arima_Course'].fillna(temp_df['user_traffic_ctype_mean_arima_Program'],inplace=True)

temp_df['user_traffic_ctype_std_arima_Course'].fillna(temp_df['user_traffic_ctype_std_arima_Degree'],inplace=True)
temp_df['user_traffic_ctype_std_arima_Course'].fillna(temp_df['user_traffic_ctype_std_arima_Program'],inplace=True)

temp_df.drop(['user_traffic_ctype_mean_arima_Degree','user_traffic_ctype_mean_arima_Program','user_traffic_ctype_std_arima_Degree',\
        'user_traffic_ctype_std_arima_Program'],\
       axis=1,inplace=True)

temp_df.rename(columns={"user_traffic_ctype_mean_arima_Course":'user_traffic_ctype_mean_arima',\
                  "user_traffic_ctype_std_arima_Course":"user_traffic_ctype_std_arima"},inplace=True)
temp_df.drop_duplicates(keep='first',inplace=True)
temp_df.head()

# # train = pd.merge(train,temp_df[["Day_No","Course_Type","user_traffic_ctype_mean_arima","user_traffic_ctype_std_arima"]],on=['Day_No','Course_Type'],how='left')
# # test = pd.merge(test,temp_df[["Day_No","Course_Type","user_traffic_ctype_mean_arima","user_traffic_ctype_std_arima"]],on=['Day_No','Course_Type'],how='left')
data = pd.merge(data,temp_df[["Day_No","Course_Type","user_traffic_ctype_mean_arima"]],on=['Day_No','Course_Type'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima,user_traffic_ctype_mean_arima
0,1,1,1,Development,Course,15398.0,7242.0,16214.0,14990.0
1,2,2,1,Development,Course,15460.0,7258.0,16098.0,15125.0
2,3,3,1,Development,Course,15454.0,7333.0,16370.0,15170.0
3,4,4,1,Development,Course,15152.0,7229.0,15934.0,14955.0
4,5,5,1,Development,Course,15261.0,6956.0,16144.0,14958.0


In [17]:
# model = ARIMA(df[df["Course_ID"]==1].user_traffic_mean,order=(3,0,2))
# results_arima = model.fit()
# print("rmse",np.sqrt(sum((df[df["Course_ID"]==1].user_traffic_mean-results_arima.fittedvalues)**2)))
# p=3,q=2 but for some svd wont converge so q=0 or 1
df = train.groupby(['Day_No','Course_ID']).agg({\
                                   'User_Traffic':{\
                                           "user_traffic_mean":'mean',
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_ID','user_traffic_mean']


temp = pd.DataFrame(columns=['Day_No'])
temp['Day_No']=np.arange(1,data.Day_No.max()+1)

vals = list(df.Course_ID.unique())
for val in vals:
    temp[str(val)]=np.nan
    
    model = ARIMA(df[df["Course_ID"]==val].user_traffic_mean,order=(2,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=test.Day_No.max())
    temp[str(val)]=round(pred,0)
    
t = temp.melt(id_vars=["Day_No"], 
              var_name="Course_ID",\
              value_name="user_traffic_id_arima")
t['Day_No'] = t['Day_No'] .astype(int)
t['Course_ID'] = t['Course_ID'].astype(int)
# train = pd.merge(train,t,on=['Day_No','Course_ID'],how='left')
# test = pd.merge(test,t,on=['Day_No','Course_ID'],how='left')
data = pd.merge(data,t,on=['Day_No','Course_ID'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_user_traffic_mean_arima,day_user_traffic_std_arima,user_traffic_dom_mean_arima,user_traffic_ctype_mean_arima,user_traffic_id_arima
0,1,1,1,Development,Course,15398.0,7242.0,16214.0,14990.0,11882.0
1,2,2,1,Development,Course,15460.0,7258.0,16098.0,15125.0,11617.0
2,3,3,1,Development,Course,15454.0,7333.0,16370.0,15170.0,12428.0
3,4,4,1,Development,Course,15152.0,7229.0,15934.0,14955.0,11798.0
4,5,5,1,Development,Course,15261.0,6956.0,16144.0,14958.0,11936.0


In [18]:
data.columns

Index(['ID', 'Day_No', 'Course_ID', 'Course_Domain', 'Course_Type',
       'day_user_traffic_mean_arima', 'day_user_traffic_std_arima',
       'user_traffic_dom_mean_arima', 'user_traffic_ctype_mean_arima',
       'user_traffic_id_arima'],
      dtype='object')

In [19]:
cols = ['ID', 
       'day_user_traffic_mean_arima', 
       'user_traffic_dom_mean_arima',
       'user_traffic_ctype_mean_arima',
       'user_traffic_id_arima']
data = data[cols]
data.head()

train=pd.merge(train,data,on='ID',how='left')
test=pd.merge(test,data,on='ID',how='left')

In [20]:
cols =['ID','Day_No','Course_ID','Course_Domain','Course_Type']
data = pd.concat([train[cols],test[cols]])
data.index = np.arange(0,len(data))
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type
0,1,1,1,Development,Course
1,2,2,1,Development,Course
2,3,3,1,Development,Course
3,4,4,1,Development,Course
4,5,5,1,Development,Course


In [21]:
arima_df = pd.DataFrame(columns=['Day_No'])
arima_df['Day_No']=np.arange(1,data.Day_No.max()+1)

df = train.groupby(['Day_No']).agg({\
                                   'Sales':{\
                                           "sale_mean":'mean',
                                            "sale_std":'std'
                                           },
                                   }).reset_index()
df.columns= ['Day_No','sale_mean','sale_std']
df.head()

model = ARIMA(df.sale_mean,order=(3,0,3))
results_arima = model.fit()
pred=results_arima.predict(start=0,end=data.Day_No.max()+1)
arima_df['day_sale_mean_arima']=round(pred,0)
#print("rmse",np.sqrt(sum((df.sale_mean-results_arima.fittedvalues)**2)))
#for sale mean arima==(3,0,0)

model = ARIMA(df.sale_std,order=(2,0,0))
results_arima = model.fit()
pred=results_arima.predict(start=0,end=data.Day_No.max()+1)
arima_df['day_sale_std_arima']=round(pred,0)
#print("rmse",np.sqrt(sum((df.sale_std-results_arima.fittedvalues)**2)))
#for sale mean arima==(2,0,0)
arima_df.head()

arima_df['Day_No']=arima_df['Day_No'].astype(int)

data = pd.merge(data,arima_df,on=['Day_No'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_sale_mean_arima,day_sale_std_arima
0,1,1,1,Development,Course,121.0,45.0
1,2,2,1,Development,Course,125.0,46.0
2,3,3,1,Development,Course,119.0,46.0
3,4,4,1,Development,Course,117.0,44.0
4,5,5,1,Development,Course,116.0,42.0


In [22]:
df = train.groupby(['Day_No','Course_Domain']).agg({\
                                   'Sales':{\
                                           "sale_mean":'mean',
                                            "sale_std":'std'
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_Domain','sale_mean','sale_std']

temp_df = data.copy()
temp_df = temp_df.groupby(["Day_No",'Course_Domain']).size().reset_index()
temp_df.columns=["Day_No",'Course_Domain','remove']
temp_df.drop(['remove'],axis=1,inplace=True)


vals = list(temp_df.Course_Domain.unique())
for val in vals:
    temp = pd.DataFrame(columns=['Day_No'])
    temp['Day_No']=np.arange(1,data.Day_No.max()+1)
    temp['Course_Domain']=val
    
    model = ARIMA(df[df["Course_Domain"]==val].sale_mean,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["sale_dom_mean_arima_"+str(val)]=round(pred,0)
    
    model = ARIMA(df[df["Course_Domain"]==val].sale_std,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["sale_dom_std_arima_"+str(val)]=round(pred,0)
    
    temp_df = pd.merge(temp_df,temp,on=['Day_No','Course_Domain'],how='outer')
    
temp_df['sale_dom_mean_arima_Business'].fillna(temp_df['sale_dom_mean_arima_Development'],inplace=True)
temp_df['sale_dom_mean_arima_Business'].fillna(temp_df['sale_dom_mean_arima_Finance & Accounting'],inplace=True)
temp_df['sale_dom_mean_arima_Business'].fillna(temp_df['sale_dom_mean_arima_Software Marketing'],inplace=True)

temp_df['sale_dom_std_arima_Business'].fillna(temp_df['sale_dom_std_arima_Development'],inplace=True)
temp_df['sale_dom_std_arima_Business'].fillna(temp_df['sale_dom_std_arima_Finance & Accounting'],inplace=True)
temp_df['sale_dom_std_arima_Business'].fillna(temp_df['sale_dom_std_arima_Software Marketing'],inplace=True)

temp_df.drop(['sale_dom_mean_arima_Development','sale_dom_mean_arima_Finance & Accounting','sale_dom_mean_arima_Software Marketing',\
        'sale_dom_std_arima_Development','sale_dom_std_arima_Finance & Accounting','sale_dom_std_arima_Software Marketing'],\
       axis=1,inplace=True)

temp_df.rename(columns={"sale_dom_mean_arima_Business":'sale_dom_mean_arima',\
                  "sale_dom_std_arima_Business":"sale_dom_std_arima"},inplace=True)
temp_df.head()

data = pd.merge(data,temp_df[["Day_No","Course_Domain","sale_dom_mean_arima","sale_dom_std_arima"]],on=['Day_No','Course_Domain'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima
0,1,1,1,Development,Course,121.0,45.0,120.0,49.0
1,2,2,1,Development,Course,125.0,46.0,125.0,49.0
2,3,3,1,Development,Course,119.0,46.0,120.0,49.0
3,4,4,1,Development,Course,117.0,44.0,117.0,47.0
4,5,5,1,Development,Course,116.0,42.0,115.0,46.0


In [23]:
df = train.groupby(['Day_No','Course_Type']).agg({\
                                   'Sales':{\
                                           "sale_mean":'mean',
                                            "sale_std":'std'
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_Type','sale_mean','sale_std']
df.head()

temp_df = data.copy()
temp_df = temp_df.groupby(["Day_No",'Course_Type']).size().reset_index()
temp_df.columns=["Day_No",'Course_Type','remove']
temp_df.drop(['remove'],axis=1,inplace=True)

vals = list(df.Course_Type.unique())
for val in vals:
    temp = pd.DataFrame(columns=['Day_No'])
    temp['Day_No']=np.arange(1,data.Day_No.max()+1)
    temp['Course_Type']=val
    
    model = ARIMA(df[df["Course_Type"]==val].sale_mean,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["sale_ctype_mean_arima_"+str(val)]=round(pred,0)
    
    model = ARIMA(df[df["Course_Type"]==val].sale_std,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=data.Day_No.max())
    temp["sale_ctype_std_arima_"+str(val)]=round(pred,0)
    
    temp_df = pd.merge(temp_df,temp,on=['Day_No','Course_Type'],how='left')

    
temp_df['sale_ctype_mean_arima_Course'].fillna(temp_df['sale_ctype_mean_arima_Degree'],inplace=True)
temp_df['sale_ctype_mean_arima_Course'].fillna(temp_df['sale_ctype_mean_arima_Program'],inplace=True)

temp_df['sale_ctype_std_arima_Course'].fillna(temp_df['sale_ctype_std_arima_Degree'],inplace=True)
temp_df['sale_ctype_std_arima_Course'].fillna(temp_df['sale_ctype_std_arima_Program'],inplace=True)

temp_df.drop(['sale_ctype_mean_arima_Degree','sale_ctype_mean_arima_Program','sale_ctype_std_arima_Degree',\
        'sale_ctype_std_arima_Program'],\
       axis=1,inplace=True)

temp_df.rename(columns={"sale_ctype_mean_arima_Course":'sale_ctype_mean_arima',\
                  "sale_ctype_std_arima_Course":"sale_ctype_std_arima"},inplace=True)
temp_df.drop_duplicates(keep='first',inplace=True)
temp_df.head()

data = pd.merge(data,temp_df[["Day_No","Course_Type","sale_ctype_mean_arima","sale_ctype_std_arima"]],on=['Day_No','Course_Type'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima,sale_ctype_mean_arima,sale_ctype_std_arima
0,1,1,1,Development,Course,121.0,45.0,120.0,49.0,114.0,41.0
1,2,2,1,Development,Course,125.0,46.0,125.0,49.0,119.0,42.0
2,3,3,1,Development,Course,119.0,46.0,120.0,49.0,114.0,42.0
3,4,4,1,Development,Course,117.0,44.0,117.0,47.0,111.0,40.0
4,5,5,1,Development,Course,116.0,42.0,115.0,46.0,109.0,39.0


In [24]:
# model = ARIMA(df[df["Course_ID"]==1].sale_mean,order=(3,0,2))
# results_arima = model.fit()
# print("rmse",np.sqrt(sum((df[df["Course_ID"]==1].sale_mean-results_arima.fittedvalues)**2)))
# p=3,q=2 but for some svd wont converge so q=0 or 1
df = train.groupby(['Day_No','Course_ID']).agg({\
                                   'Sales':{\
                                           "sale_mean":'mean',
                                           }
                                   }).reset_index().fillna(0)
df.columns= ['Day_No','Course_ID','sale_mean']


temp = pd.DataFrame(columns=['Day_No'])
temp['Day_No']=np.arange(1,data.Day_No.max()+1)

vals = list(df.Course_ID.unique())
for val in vals:
    temp[str(val)]=np.nan
    
    model = ARIMA(df[df["Course_ID"]==val].sale_mean,order=(3,0,0))
    results_arima = model.fit()
    pred=results_arima.predict(start=0,end=test.Day_No.max())
    temp[str(val)]=round(pred,0)
    
t = temp.melt(id_vars=["Day_No"], 
              var_name="Course_ID",\
              value_name="sale_id_arima")
t['Day_No'] = t['Day_No'] .astype(int)
t['Course_ID'] = t['Course_ID'].astype(int)
# train = pd.merge(train,t,on=['Day_No','Course_ID'],how='left')
# test = pd.merge(test,t,on=['Day_No','Course_ID'],how='left')
data = pd.merge(data,t,on=['Day_No','Course_ID'],how='left')
data.head()

,ID,Day_No,Course_ID,Course_Domain,Course_Type,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima,sale_ctype_mean_arima,sale_ctype_std_arima,sale_id_arima
0,1,1,1,Development,Course,121.0,45.0,120.0,49.0,114.0,41.0,85.0
1,2,2,1,Development,Course,125.0,46.0,125.0,49.0,119.0,42.0,84.0
2,3,3,1,Development,Course,119.0,46.0,120.0,49.0,114.0,42.0,83.0
3,4,4,1,Development,Course,117.0,44.0,117.0,47.0,111.0,40.0,82.0
4,5,5,1,Development,Course,116.0,42.0,115.0,46.0,109.0,39.0,84.0


In [25]:
cols = ['ID', 
       'day_sale_mean_arima', 'day_sale_std_arima', 'sale_dom_mean_arima',
       'sale_dom_std_arima', 'sale_ctype_mean_arima', 'sale_ctype_std_arima',
       'sale_id_arima']
data = data[cols]
data.head()

,ID,day_sale_mean_arima,day_sale_std_arima,sale_dom_mean_arima,sale_dom_std_arima,sale_ctype_mean_arima,sale_ctype_std_arima,sale_id_arima
0,1,121.0,45.0,120.0,49.0,114.0,41.0,85.0
1,2,125.0,46.0,125.0,49.0,119.0,42.0,84.0
2,3,119.0,46.0,120.0,49.0,114.0,42.0,83.0
3,4,117.0,44.0,117.0,47.0,111.0,40.0,82.0
4,5,116.0,42.0,115.0,46.0,109.0,39.0,84.0


In [26]:
train=pd.merge(train,data,on='ID',how='left')
test=pd.merge(test,data,on='ID',how='left')

In [27]:
# train['Day_No'] = train["Day_No"].astype(int)
# test['Day_No'] = test["Day_No"].astype(int)

# train['day_sin'] = abs(np.sin(2*np.pi*train['Day_No']/train['Day_No'].max()))
# test['day_sin'] = abs(np.sin(2*np.pi*test['Day_No']/test['Day_No'].max()))

# train['total_promotion'] = train['Long_Promotion']+train['Short_Promotion']
# train['shortBYLong'] = train['Short_Promotion']/(1+train['Long_Promotion'])

# test['total_promotion'] = test['Long_Promotion']+test['Short_Promotion']
# test['shortBYLong'] = test['Short_Promotion']/(1+test['Long_Promotion'])

In [28]:
# train['salebytraffic'] = (1-train['Competition_Metric'])*train['sale_id_arima']/train['User_Traffic']
# test['salebytraffic'] = (1-test['Competition_Metric'])*test['sale_id_arima']/test['User_Traffic']

# train['salebytraffic'] = (1-train['Competition_Metric'])*train['user_traffic_id_arima']
# test['salebytraffic'] = (1-test['Competition_Metric'])*test['user_traffic_id_arima']


# train['daytraffic__comp'] = train['day_user_traffic_mean_arima']/train['Competition_Metric']
# test['daytraffic__comp'] = test['day_user_traffic_mean_arima']/test['Competition_Metric']

# train['salebytraffic'] = train['sale_id_arima']/train['user_traffic_id_arima']
# test['salebytraffic'] = test['sale_id_arima']/test['user_traffic_id_arima']

# train['comp_metric_traffic_id'] = train['Competition_Metric']*train['day_user_traffic_mean_arima']
# test['comp_metric_traffic_id'] = test['Competition_Metric']*test['day_user_traffic_mean_arima']

# train['comp_metric_traffic_dom'] = train['Competition_Metric']*train['user_traffic_dom_mean_arima']
# test['comp_metric_traffic_dom'] = test['Competition_Metric']*test['user_traffic_dom_mean_arima']

In [29]:
cols = ['Course_ID', 'Course_Domain', 'Course_Type','Public_Holiday']
for col in cols:
    train[col].astype(object)
train.drop(['ID','Day_No'],axis=1,inplace=True)    
train = pd.get_dummies(train,columns=['Course_Domain','Course_Type','Course_ID'])
train.fillna(train.mean(),inplace=True)


cols = ['Course_Domain', 'Course_Type','Public_Holiday']
for col in cols:
    test[col].astype(object)
test.index = test.ID
test.drop(['ID','Day_No'],axis=1,inplace=True)    
test = pd.get_dummies(test,columns=['Course_Domain', 'Course_Type','Course_ID'])
test.fillna(test.mean(),inplace=True)

In [30]:
test.shape,train.shape

((36000, 622), (512087, 624))

In [53]:
test_ =test.copy()

X = train[[col for col in train.columns if col not in (['Sales','day_sale_mean_arima', 'day_sale_std_arima', 'sale_dom_mean_arima',
       'sale_dom_std_arima', 'sale_ctype_mean_arima', 'sale_ctype_std_arima','User_Traffic',
       'sale_id_arima', 'Course_Domain_Business',
          'Course_Domain_Development', 'Course_Domain_Finance & Accounting',
         'Course_Domain_Software Marketing'
                                                       ])]]
test_.drop(['day_sale_mean_arima', 'day_sale_std_arima', 'sale_dom_mean_arima',
       'sale_dom_std_arima', 'sale_ctype_mean_arima', 'sale_ctype_std_arima',
       'sale_id_arima', 'Course_Domain_Business',
          'Course_Domain_Development', 'Course_Domain_Finance & Accounting',
         'Course_Domain_Software Marketing'],axis=1,inplace=True)
y = train.Sales

kf = KFold(n_splits=10)
test_preds = np.zeros((len(test_),1))

lr = LinearRegression()
rfc = RandomForestRegressor()
lgbm = lightgbm.LGBMRegressor(n_estimators=1000,\
                             max_depth=8,\
                             #colsample_bytree=0.9
                             )

cols_name = ['lgbm','rfc']
result2 = pd.DataFrame(columns=cols_name)
feature_imp = np.zeros((len(X.columns),1))
i=0
classfiers = [lgbm,rfc]
for clf in classfiers:
    test_preds = np.zeros((len(test),1))
    scores = []
    for train_idx,val_idx in kf.split(X,y):
        X_train,X_val = X.iloc[train_idx],X.iloc[val_idx]
        y_train,y_val = y.iloc[train_idx],y.iloc[val_idx]

        clf.fit(X_train,y_train)
        pred = clf.predict(X_val)
        #feature_imp += clf.feature_importances_.reshape(-1,1)
        pred = np.where(pred<0,0,pred)
        score = np.sqrt(mean_squared_log_error(y_val,pred))
        scores.append(score)
        print(1000*score)

        test_pred = clf.predict(test_).reshape(-1,1)
        test_pred = np.where(test_pred<0,0,test_pred)
        test_preds += test_pred
        
    test_preds = test_preds/10
    result2.iloc[:,i]=test_preds[:,0]
    print(clf)
    print(round(1000*np.mean(scores),2),end="\n")
    i+=1
    


# print("\nmean score= ",1000*np.mean(scores))

# feature_imp = feature_imp/10
# feature_imp = pd.DataFrame(feature_imp, columns=['Value'])
# feature_imp['columns']=X.columns
# feature_imp = feature_imp.sort_values(by='Value')

226.13613178886064
217.63156267524477
229.15200647012347
223.39060684782953
263.4941456241916
244.08217780788414
229.0630408128372
244.41296875792077
215.64803516704356
226.0082111791932
LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=8,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
231.9
236.52921702946753
225.11105717720739
241.21610127987947
236.99657465668488
272.83991130677043
253.6657824522915
248.3825350363277
263.3558990160693
224.90384956092402
242.11726221057495
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impuri

In [ ]:
# feature_imp = feature_imp/10
# feature_imp = pd.DataFrame(feature_imp, columns=['Value'])
# feature_imp['columns']=X.columns
# feature_imp = feature_imp.sort_values(by='Value')
# feature_imp[-10:]

In [54]:
test_nn = test.copy()
cols = [\
       'day_sale_mean_arima', 'day_sale_std_arima', 'sale_dom_mean_arima',
       'sale_dom_std_arima', 'sale_ctype_mean_arima', 'sale_ctype_std_arima',
       'sale_id_arima']
X_nn = train[cols]

test_nn = test_nn[cols]

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasRegressor



clf = Sequential([
                    Dense(32, activation='relu', input_shape=[len(X_nn.columns)]),
                    Dense(64, activation='relu'),
                    Dense(96, activation='relu'),
                    Dense(32, activation='relu'),
                    Dense(1, activation='linear')
                  ])
clf.compile(loss='mse',optimizer='adam',metrics=[tf.keras.losses.msle])

history = clf.fit(X_nn,y,validation_split=0.1,epochs=200,batch_size=1000,verbose=2)
test_pred = clf.predict(test_nn)
result2['keras'] = test_pred

Using TensorFlow backend.



Train on 460878 samples, validate on 51209 samples
Epoch 1/200
 - 1s - loss: 1145.4129 - mean_squared_logarithmic_error: 0.1100 - val_loss: 1047.9676 - val_mean_squared_logarithmic_error: 0.0941
Epoch 2/200
 - 1s - loss: 990.6519 - mean_squared_logarithmic_error: 0.0969 - val_loss: 1012.3100 - val_mean_squared_logarithmic_error: 0.0918
Epoch 3/200
 - 1s - loss: 978.0317 - mean_squared_logarithmic_error: 0.0963 - val_loss: 1030.8835 - val_mean_squared_logarithmic_error: 0.0906
Epoch 4/200
 - 1s - loss: 965.5801 - mean_squared_logarithmic_error: 0.0958 - val_loss: 977.2297 - val_mean_squared_logarithmic_error: 0.0912
Epoch 5/200
 - 1s - loss: 946.0089 - mean_squared_logarithmic_error: 0.0948 - val_loss: 954.2832 - val_mean_squared_logarithmic_error: 0.0903
Epoch 6/200
 - 1s - loss: 921.3081 - mean_squared_logarithmic_error: 0.0933 - val_loss: 930.4519 - val_mean_squared_logarithmic_error: 0.0907
Epoch 7/200
 - 1s - loss: 899.1474 - mean_squared_logarithmic_error: 0.0914 - val_loss: 917.4

Epoch 58/200
 - 1s - loss: 631.6891 - mean_squared_logarithmic_error: 0.0586 - val_loss: 676.1439 - val_mean_squared_logarithmic_error: 0.0606
Epoch 59/200
 - 1s - loss: 633.4414 - mean_squared_logarithmic_error: 0.0588 - val_loss: 652.8896 - val_mean_squared_logarithmic_error: 0.0564
Epoch 60/200
 - 1s - loss: 620.6326 - mean_squared_logarithmic_error: 0.0579 - val_loss: 642.2048 - val_mean_squared_logarithmic_error: 0.0562
Epoch 61/200
 - 1s - loss: 626.0476 - mean_squared_logarithmic_error: 0.0580 - val_loss: 655.5211 - val_mean_squared_logarithmic_error: 0.0577
Epoch 62/200
 - 1s - loss: 619.7134 - mean_squared_logarithmic_error: 0.0573 - val_loss: 645.2299 - val_mean_squared_logarithmic_error: 0.0561
Epoch 63/200
 - 1s - loss: 619.3127 - mean_squared_logarithmic_error: 0.0572 - val_loss: 638.6439 - val_mean_squared_logarithmic_error: 0.0551
Epoch 64/200
 - 1s - loss: 613.1611 - mean_squared_logarithmic_error: 0.0562 - val_loss: 652.5659 - val_mean_squared_logarithmic_error: 0.0562

Epoch 116/200
 - 1s - loss: 543.9379 - mean_squared_logarithmic_error: 0.0509 - val_loss: 589.9986 - val_mean_squared_logarithmic_error: 0.0492
Epoch 117/200
 - 1s - loss: 546.8632 - mean_squared_logarithmic_error: 0.0512 - val_loss: 580.4753 - val_mean_squared_logarithmic_error: 0.0508
Epoch 118/200
 - 1s - loss: 543.3374 - mean_squared_logarithmic_error: 0.0509 - val_loss: 590.8295 - val_mean_squared_logarithmic_error: 0.0499
Epoch 119/200
 - 1s - loss: 545.2896 - mean_squared_logarithmic_error: 0.0508 - val_loss: 583.3812 - val_mean_squared_logarithmic_error: 0.0513
Epoch 120/200
 - 1s - loss: 546.1656 - mean_squared_logarithmic_error: 0.0509 - val_loss: 571.0611 - val_mean_squared_logarithmic_error: 0.0492
Epoch 121/200
 - 1s - loss: 542.3978 - mean_squared_logarithmic_error: 0.0508 - val_loss: 579.9932 - val_mean_squared_logarithmic_error: 0.0522
Epoch 122/200
 - 1s - loss: 540.7043 - mean_squared_logarithmic_error: 0.0510 - val_loss: 577.3027 - val_mean_squared_logarithmic_error:

Epoch 173/200
 - 1s - loss: 507.6555 - mean_squared_logarithmic_error: 0.0480 - val_loss: 529.4923 - val_mean_squared_logarithmic_error: 0.0472
Epoch 174/200
 - 1s - loss: 511.4190 - mean_squared_logarithmic_error: 0.0483 - val_loss: 540.6174 - val_mean_squared_logarithmic_error: 0.0480
Epoch 175/200
 - 1s - loss: 507.6822 - mean_squared_logarithmic_error: 0.0482 - val_loss: 544.1302 - val_mean_squared_logarithmic_error: 0.0471
Epoch 176/200
 - 1s - loss: 506.3687 - mean_squared_logarithmic_error: 0.0481 - val_loss: 543.9519 - val_mean_squared_logarithmic_error: 0.0473
Epoch 177/200
 - 1s - loss: 503.8732 - mean_squared_logarithmic_error: 0.0479 - val_loss: 528.0593 - val_mean_squared_logarithmic_error: 0.0463
Epoch 178/200
 - 1s - loss: 504.4954 - mean_squared_logarithmic_error: 0.0482 - val_loss: 545.3863 - val_mean_squared_logarithmic_error: 0.0467
Epoch 179/200
 - 1s - loss: 504.8021 - mean_squared_logarithmic_error: 0.0478 - val_loss: 536.7650 - val_mean_squared_logarithmic_error:

In [55]:
result2['Sales'] = result2.sum(axis=1)/3
result2 = round(result2,0)
result2.head()

,lgbm,rfc,keras,Sales
0,101.0,107.0,119.0,109.0
1,126.0,116.0,96.0,113.0
2,107.0,100.0,91.0,100.0
3,110.0,102.0,89.0,101.0
4,68.0,69.0,67.0,68.0


In [56]:
temp = test.copy()
temp['Sales'] = result2.Sales.values
temp = temp[['Sales']]
temp.to_csv('final.csv')
temp.head()

,Sales
ID,
883,109.0
884,113.0
885,100.0
886,101.0
887,68.0
